Initial includes and some functions.

In [1]:
from dist_tools import *

%matplotlib inline

fips, epsg, seats, usps = 42, 3364, 18, "pa"
years = [2000, 2004, 2008, 2012]

In [2]:
cols_00_08 = ["D00", "R00", "D04", "R04", "D08", "R08", "DNorm", "RNorm"]
cols_12    = ["D12", "R12"]
cols = cols_00_08 + cols_12

The 2000-2008 data is pretty clean -- just load it and change the column names.

In [3]:
votes = pd.read_csv("/media/jsaxon/brobdingnag/data/el_dv/pa/pa_final.tab",
                    sep = "\t", low_memory = False)

votes.rename(columns = {"uspdv2000" : "D00", "uspdv2004" : "D04", "uspdv2008" : "D08", "uspdv2012" : "D12",
                        "usprv2000" : "R00", "usprv2004" : "R04", "usprv2008" : "R08", "usprv2012" : "R12",
                        "ndv" : "DNorm", "nrv" : "RNorm", "geoid10" : "geoid"}, inplace = True)

votes = votes[["geoid"] + cols_00_08]

Now we begin on the messy business of PA 2012 -- we have to clean and then fuzzy match the data to the precincts, with a hard match on county.

In [4]:
con = psycopg2.connect(database = "census", user = user, password = passwd,
                       host = "saxon.harris.uchicago.edu", port = 5432)

pa_name_query = """SELECT vtd.state, co.name cname, vtd.county cid, vtd, vtd.geoid,
                     REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(
                     REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(
                     REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(
                     REGEXP_REPLACE(REGEXP_REPLACE(
                       UPPER(vtd.name), '\.', '', 'g'), ' 0', ' ', 'g'), 'VTD ', 'X-'), ' VOTING DISTRICT', ''),
                                        ' - ', ' ', 'g'), ' WD ', ' W-', 'g'), ',', ''), ' DIST ', ' D-'),
                                        ' PCT ', ' P-'), ' BORO', ''), ' CITY', ''), ' TWP', ''), ' ED ', ' X-'),
                                        ' DIST PCT', ' D-')
                     vname
                   FROM vtd_2010 vtd
                   JOIN counties_2016 co ON
                     co.state  = vtd.state AND
                     co.county = vtd.county
                   WHERE
                     vtd.state = 42
                   ORDER BY cid, vname;"""

prdf = pd.read_sql(pa_name_query, con)

pa_co = prdf[["cname", "cid"]].drop_duplicates().reset_index()
pa_co["county"] = pa_co.cname.str.upper()
pa_co = pa_co[["cid", "county"]]

votes_2012 = pd.read_csv("/media/jsaxon/brobdingnag/data/el_dv/precincts/PA_2012.tab", 
                         delimiter = "\t", low_memory = False)
votes_2012.rename(columns={"precinct_code" : "vtd", 
                           "g2012_USP_dv"  : "D12", 
                           "g2012_USP_rv"  : "R12"}, 
                           inplace = True)
votes_2012 = votes_2012.merge(pa_co, on = "county")
votes_2012 = votes_2012[["county", "cid", "precinct", "D12", "R12"]]

In order to complete the matches, we also have to do a bit of fixing by hand.

In [5]:
pct = votes_2012.precinct.str.upper()
pct = pct.str.replace(" BORO", "")
pct = pct.str.replace(" CITY", "")
pct = pct.str.replace(" TWP", "")
pct = pct.str.replace(" - ", " ")
pct = pct.str.replace(".", "")
pct = pct.str.replace(",", "")
votes_2012.precinct = pct

prdf.loc[prdf.cid == 7,  ["vname"]] = prdf.loc[prdf.cid == 7,  ["vname"]].replace({"vname" : {' [0-9] ' : " "}}, regex = True)
prdf.loc[prdf.cid == 95, ["vname"]] = prdf.loc[prdf.cid == 95, ["vname"]].replace({"vname" : {r'LOWER SAUCON X-([0-9]) [A-Z ]*' :
                                                                                              r'LOWER SAUCON X-\1'}}, regex = True)
prdf.loc[prdf.cid == 37, ["vname"]] = prdf.loc[prdf.cid == 37, ["vname"]].replace({"vname" : {r'BLOOMSBURG [TOWN D-]*([1-4])[STNDH WARD]*([ X\-1-2]*)' :
                                                                                              r'BLOOMSBURG W-\1 \2'}}, regex = True)
prdf.loc[prdf.cid == 37, ["vname"]] = prdf.loc[prdf.cid == 37, ["vname"]].replace({"vname" : {r'BERWICK ([1-4])[STNDH WARD].*' :
                                                                                              r'BERWICK W-\1'}}, regex = True)

prdf.loc[(prdf.cid == 11) & (prdf.vname == "MOUNT PENN D-1"), "vname"] = "MT PENN D-1"

prdf.loc[(prdf.cid == 25) & (prdf.vname == "EAST PENN D-NORTH"), "vname"] = "EAST PENN X-NORTH"

prdf.loc[(prdf.cid == 107) & (prdf.vname == "NORTH MANHEIM D-PCT 1"),       "vname"] = "NORTH MANHEIM P-1"
prdf.loc[(prdf.cid == 107) & (prdf.vname == "NORTH MANHEIM D-MELLOTS P-2"), "vname"] = "NORTH MANHEIM P-2"
prdf.loc[(prdf.cid == 107) & (prdf.vname == "NORTH MANHEIM D-ADAMSDALE"),   "vname"] = "NORTH MANHEIM P-3"

votes_2012.loc[(votes_2012.cid == 25) & (votes_2012.precinct == "SUMMIT HILL D-3 X-1"), "precinct"] = "SUMMIT HILL W-3 D-1"
votes_2012.loc[(votes_2012.cid == 25) & (votes_2012.precinct == "SUMMIT HILL D-3 X-2"), "precinct"] = "SUMMIT HILL W-3 D-2"

Hurrah for fuzzy matching!!

In [6]:
votes_2012['vname'] = "NOMATCH"
for cid in list(pa_co["cid"]):
    
    # Alternately:  .apply(lambda x: process.extractOne(x, prdf[prdf.cid == cid]["vname"])[0])
    votes_2012.loc[votes_2012.cid == cid, 'vname'] = votes_2012[votes_2012.cid == cid]['precinct'].apply(lambda x: jf_jw_match(x, prdf[prdf.cid == cid]["vname"])[0])
    

In [7]:
votes_2012 = votes_2012.merge(prdf, on = ["cid", "vname"])[["state", "cid", "cname", "precinct", "D12", "R12", "geoid"]]

Get the tract to precinct mapping directly from postgres.

In [8]:
pr_tr_query = """SELECT
                   rn.rn, tr.state, tr.county cid, UPPER(co.name) county, tr.tract, 
                   pr.vtd, pr.name, pr.geoid
                 FROM census_tracts_2015 AS tr
                 JOIN vtd_2010 AS pr ON
                   pr.state  = tr.state  AND
                   pr.county = tr.county AND
                   ST_Covers(tr.geom, ST_Centroid(pr.geom))
                 JOIN (SELECT state, county, tract,
                              row_number() over (PARTITION BY state ORDER BY county, tract NULLS LAST) - 1 as rn
                       FROM census_tracts_2015) rn ON
                   tr.state  = rn.state  AND
                   tr.county = rn.county AND
                   tr.tract  = rn.tract
                 JOIN counties_2016 co ON
                   tr.state  = co.state AND
                   tr.county = co.county
                 WHERE tr.state = {}
                 ORDER BY tr.state, tr.county, tr.tract, pr.vtd;"""

con = psycopg2.connect(database = "census", user = user, password = passwd,
                         host = "saxon.harris.uchicago.edu", port = 5432)

# trprdf = pd.read_sql(pr_tr_query.format(fips), con)
# trprdf.to_csv("pa_trpr.csv", index = False)

pr_tr = pd.read_csv("pa_trpr.csv")

Merge everything together.

In [9]:
tr_votes = pr_tr.merge(votes, on = "geoid")
tr_votes = tr_votes.merge(votes_2012, on = "geoid")
tr_votes = tr_votes.groupby("rn")[cols].sum()
votes = tr_votes.fillna(0.0).astype(int)

In [10]:
votes.to_csv("pa_votes.csv")